In [1]:
# Install dependencies

import sys
!{sys.executable} -m pip install hdfs

In [2]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/TrabalhoPL/gold'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# create gold database
spark.sql(
    """
    DROP DATABASE IF EXISTS Projeto_gold CASCADE
    """
)
spark.sql(
    """
    create database Projeto_gold location 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db'
    """
)


DataFrame[]

In [2]:
from pyspark.sql.functions import substring, avg, sum

# read air_quality from the silver tables
hdfs_path = "hdfs://hdfs-nn:9000/TrabalhoPL/silver/Projeto.db/Tabela_Agua"

agua = spark\
             .read\
             .load(hdfs_path)

agua.show()

+---------+-----------+-----------+-----------+--------------------+------------+------------+------------+-------------+----+
|Sample_Id|Sample_Site|Sample_Date|Sample_Time|             Analyte|      Status|Final_Result|       Units| Stream_Group| Ano|
+---------+-----------+-----------+-----------+--------------------+------------+------------+------------+-------------+----+
| C-175259|       E16I| 1997-01-02|       8:32|Solids, Total Sus...|Desconhecido|           3|        mg/L|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|                  pH|Desconhecido|        6.73|Desconhecida|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|         Temperature|Desconhecido|           1|           C|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|      Apparent Color|Desconhecido|           6|          CU|       Esopus|1997|
| C-175259|       E16I| 1997-01-02|       8:32|     Scent Character|Desconhecido|           0|Desconhecida|    

In [3]:
from pyspark.sql.functions import avg
gold_agua = agua \
    .groupBy("Ano", "Analyte", "Stream_Group")\
    .agg(
        avg(agua.Final_Result).alias("Media")
    ) \
   
gold_agua.toPandas()


,Ano,Analyte,Stream_Group,Media
0,1997,"Copper, Total (Undigested)",Catskill Daily,4.000000
1,1998,Chloride,Cannonsville,13.690735
2,1998,"Antimony, Total (Undigested)",Cannonsville,0.160000
3,1998,"Chromium, Total (Undigested)",Lower Esopus,NaN
4,1999,Scent Character,Cannonsville,0.170732
...,...,...,...,...
14750,2020,"Magnesium, Dissolved",East Branch,9.108750
14751,2013,Ammonia (as N),Neversink,0.061818
14752,2020,Sulfate (as SO4),Esopus,2.992857
14753,2013,Sulfate (as SO4),Neversink,4.330769


In [4]:
# create air_quality table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto_gold.Tabela_Agua_3
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Agua_3 (
        Ano INT,
        Analyte VARCHAR(250), 
        Stream_Group VARCHAR(250),
        Media DOUBLE
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua_3/'
    """
)
        

DataFrame[]

In [5]:
# write to delta table
gold_agua \
    .write  \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua_3/")

In [6]:
# check the results in the table
spark.table("Projeto_gold.Tabela_Agua_3").toPandas()


,Ano,Analyte,Stream_Group,Media
0,1997,"Copper, Total (Undigested)",Catskill Daily,4.000000
1,1998,Chloride,Cannonsville,13.690735
2,1998,"Antimony, Total (Undigested)",Cannonsville,0.160000
3,1998,"Chromium, Total (Undigested)",Lower Esopus,NaN
4,1999,Scent Character,Cannonsville,0.170732
...,...,...,...,...
14750,2020,"Magnesium, Dissolved",East Branch,9.108750
14751,2013,Ammonia (as N),Neversink,0.061818
14752,2020,Sulfate (as SO4),Esopus,2.992857
14753,2013,Sulfate (as SO4),Neversink,4.330769


In [7]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua_3/`
""").show()

++
||
++
++



In [8]:
spark.sql("""
DROP TABLE IF EXISTS Projeto_gold.Tabela_Agua_Presto_3 
""").show()

spark.sql("""
    CREATE EXTERNAL TABLE Projeto_gold.Tabela_Agua_Presto_3 (
        Ano INT,
        Analyte VARCHAR(250), 
        Stream_Group VARCHAR(250),
        Media DOUBLE
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/gold/Projeto_gold.db/Tabela_Agua_3/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

